In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.io import show, output_notebook
output_notebook()
 

from scripts_viz.visualization_utils import *
from scripts_viz.visualization_utils import TTQcolor

Loading BokehJS ...

In [2]:
datafolder = '../data/viz_data/'
experiment = 'benchmarks_shuffle_imp'

viz = pd.read_csv(datafolder+experiment+'/'+experiment+'.csv')

In [3]:
pd.set_option("display.max_rows", 101)

In [4]:
#viz.transpose()['model_filename'].str.split('.pkl')

In [5]:
viz = viz.transpose()
viz.columns = viz.transpose()['model_filename'].apply(lambda x:x.split('.pkl')[0])
viz

model_filename,shuffle_imp__RandomForestClassifier_190727_131,shuffle_imp__SGDClassifier_190727_130
Run ID,acad0645ae7b4647b36d3a8d26fa2aff,905bbbb585f1436fb429f9e5ad7359b1
Name,shuffle_imp_RandomForestClassifier,shuffle_imp_SGDClassifier
Source Type,LOCAL,LOCAL
Source Name,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...
User,david,david
Status,FINISHED,FINISHED
alpha,NaN,0.0001
average,NaN,False
bootstrap,True,NaN
class_weight,balanced,None


In [6]:
viz.columns

Index(['shuffle_imp__RandomForestClassifier_190727_131', 'shuffle_imp__SGDClassifier_190727_130'], dtype='object', name='model_filename')

In [7]:
#retrieving dict viz and organizing it in a macrodictionary
viz_dicts = dict(zip(list(viz.columns), [pd.read_pickle(viz.loc['model_filepath', modelname].replace('models', 'viz_data').replace('.pkl','_viz.pkl')) for modelname in viz.columns]))

In [28]:
val_roc = plot_rocs([viz_dicts[model]['validation'] for model in viz.columns], label = list(viz.columns), title_lab = 'Validation performance', 
               p_width=500, p_height=500, colors = [TTQcolor['azureBlue'], TTQcolor['richOrange']])

show(val_roc)

In [21]:
test_roc = plot_rocs([viz_dicts[model]['testing'] for model in viz.columns], label = list(viz.columns), title_lab = 'Test performance',
               p_width=600, p_height=600, colors = [TTQcolor['azureBlue'], TTQcolor['richOrange']])

show(test_roc)

In [10]:
viz.loc['tp_rate'] = viz.loc['test_tp']/(viz.loc['test_tp'] + viz.loc['test_fn'])
viz.loc['tn_rate'] = viz.loc['test_tn']/(viz.loc['test_tn'] + viz.loc['test_fp'])

In [11]:
viz

model_filename,shuffle_imp__RandomForestClassifier_190727_131,shuffle_imp__SGDClassifier_190727_130
Run ID,acad0645ae7b4647b36d3a8d26fa2aff,905bbbb585f1436fb429f9e5ad7359b1
Name,shuffle_imp_RandomForestClassifier,shuffle_imp_SGDClassifier
Source Type,LOCAL,LOCAL
Source Name,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...
User,david,david
Status,FINISHED,FINISHED
alpha,NaN,0.0001
average,NaN,False
bootstrap,True,NaN
class_weight,balanced,None


In [22]:
params = ['tp_rate', 'tn_rate', 'test_auc', 'val_auc']
models = list(viz.columns)
spider = spiderWebChart(models, params, 
                    [viz.loc[params, m] for m in models], 
                    colors=[TTQcolor['richOrange'], TTQcolor['azureBlue']], text_size='12pt')

show(spider)

In [23]:
folds = histfolds(list(viz.columns), [fold for fold in viz.index if 'val_auc_fold_' in fold], viz, plot_w=600)
show(folds)

In [26]:
from bokeh.layouts import gridplot

l = gridplot([[val_roc, test_roc],[spider, folds]])
show(l)

In [34]:
len(data['val_auc_fold_1'])

2

In [27]:
[TTQcolor['richOrange']]*4 #len(data[metrics][0])

['#FB8C36', '#FB8C36', '#FB8C36', '#FB8C36']

In [ ]:
#comparing random forests
#max_depth
#max_features ---------- handling sqrt
#max_leaf_nodes
#min_samples_leaf
#min_samples_split
#n_estimators
#bootstrap
#max_depth


#comparing SGD